# AGE

In [33]:
import kagglehub


path = kagglehub.dataset_download("jangedoo/utkface-new")
print("Path to dataset files:", path)

Path to dataset files: /Users/leighchejaikarran/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1


In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

In [2]:
dataset_dir = os.path.join("/Users/Leighchejaikarran/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1", "UTKFace")

In [ ]:
image_filenames = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]

# Define age range buckets
def age_to_class(age):
    if age < 10:
        return 0  # 0–9
    elif age < 20:
        return 1  # 10–19
    elif age < 30:
        return 2  # 20–29
    elif age < 40:
        return 3  # 30–39
    elif age < 50:
        return 4  # 40–49
    elif age < 60:
        return 5  # 50–59
    elif age < 70:
        return 6  # 60–69
    elif age < 80:
        return 7  # 70–79
    else:
        return 8  # 80–89

# Create labeled data
data = []
for filename in image_filenames:
    try:
        age = int(filename.split('_')[0])
        if age < 90:
            age_class = age_to_class(age)
            data.append({'filename': filename, 'age': age, 'age_class': age_class})
    except:
        continue  

df = pd.DataFrame(data)
print(df.head())


                                filename  age  age_class
0   9_1_2_20161219204347420.jpg.chip.jpg    9          0
1  36_0_1_20170117163203851.jpg.chip.jpg   36          3
2  86_1_0_20170120225751953.jpg.chip.jpg   86          8
3  26_1_0_20170116171048641.jpg.chip.jpg   26          2
4   1_1_2_20161219154612988.jpg.chip.jpg    1          0


# Preprocessing

In [ ]:
img_size = 128
batch_size = 32
num_classes = 9

def preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, img_size, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(tf.cast(label, tf.int32), depth=num_classes)
    return image, label

def create_dataset(df):
    filepaths = [os.path.join(dataset_dir, fname) for fname in df['filename']]
    labels = df['age_class'].values
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df)

# Building Classification Model

In [11]:
base_model = MobileNetV2(input_shape=(img_size, img_size, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')  # Multi-class classification
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,113 (9.24 MB)

 Trainable params: 165,129 (645.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Train Model

In [12]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 54s 87ms/step - accuracy: 0.4044 - loss: 1.6193 - val_accuracy: 0.4730 - val_loss: 1.3257
Epoch 2/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 53s 90ms/step - accuracy: 0.4775 - loss: 1.3459 - val_accuracy: 0.4853 - val_loss: 1.2943
Epoch 3/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 55s 93ms/step - accuracy: 0.4887 - loss: 1.3091 - val_accuracy: 0.4977 - val_loss: 1.2794
Epoch 4/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 56s 94ms/step - accuracy: 0.4982 - loss: 1.2635 - val_accuracy: 0.4998 - val_loss: 1.2642
Epoch 5/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 54s 92ms/step - accuracy: 0.5140 - loss: 1.2424 - val_accuracy: 0.4955 - val_loss: 1.2822


# Fine Tuning

In [13]:
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
589/589 ━━━━━━━━━━━━━━━━━━━━ 76s 120ms/step - accuracy: 0.3526 - loss: 1.8968 - val_accuracy: 0.4768 - val_loss: 1.3499
Epoch 2/3
589/589 ━━━━━━━━━━━━━━━━━━━━ 73s 123ms/step - accuracy: 0.4577 - loss: 1.3989 - val_accuracy: 0.4885 - val_loss: 1.3163
Epoch 3/3
589/589 ━━━━━━━━━━━━━━━━━━━━ 73s 125ms/step - accuracy: 0.4924 - loss: 1.2768 - val_accuracy: 0.5011 - val_loss: 1.3016


In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("age_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/lq/90xs6mj17d73071t7twq5nmr0000gn/T/tmpbz_qreo6/assets


INFO:tensorflow:Assets written to: /var/folders/lq/90xs6mj17d73071t7twq5nmr0000gn/T/tmpbz_qreo6/assets


Saved artifact at '/var/folders/lq/90xs6mj17d73071t7twq5nmr0000gn/T/tmpbz_qreo6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  13324019312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13329884256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13329883728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13329884960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13329884608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13303884432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13341868096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13341868448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13303876416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13303875536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133418

W0000 00:00:1753736521.071625  436693 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1753736521.071673  436693 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-28 23:02:01.072061: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/lq/90xs6mj17d73071t7twq5nmr0000gn/T/tmpbz_qreo6
2025-07-28 23:02:01.080909: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-28 23:02:01.080927: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/lq/90xs6mj17d73071t7twq5nmr0000gn/T/tmpbz_qreo6
I0000 00:00:1753736521.175336  436693 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-28 23:02:01.193348: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-28 23:02:01.856853: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder

In [16]:
model.save("age_model.h5")
